## Data Preparation

### Imports

In [53]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
  AutoTokenizer,
  AutoConfig,
  AutoModelForSequenceClassification,
  DataCollatorWithPadding,
  TrainingArguments,
  Trainer
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

import evaluate
import torch
import numpy as np


### Loading raw model

In [54]:
model_checkpoint = 'distilbert-base-uncased'

# Define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

# Generate classification model from checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
  model_checkpoint, num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Loading dataset

In [55]:
dataset = load_dataset("imdb")
# dataset
dataset['train'] = dataset['train'].select(range(0, 100))
dataset['test'] = dataset['test'].select(range(0, 100))
dataset['unsupervised'] = dataset['unsupervised'].select(range(0, 100))
dataset

Found cached dataset imdb (C:/Users/Pawel/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)

100%|██████████| 3/3 [00:00<00:00, 257.96it/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

### Preprocessing data

In [56]:
# Create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Create tokenize function
def tokenize_function(examples):
  # extract text
  text = examples["text"]
  
  # Tokenize and truncate text
  tokenizer.truncation_side = "left"
  tokenized_inputs = tokenizer(
    text,
    return_tensors="np",
    truncation=True,
    max_length=512,
  )
  
  return tokenized_inputs

# Add pad tokens if none exists
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({"pad_token": "[PAD]"})
  model.resize_model_embeddings(len(tokenizer))
  
# Tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Loading cached processed dataset at C:\Users\Pawel\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-114fbcb44d4e74d7.arrow



Loading cached processed dataset at C:\Users\Pawel\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-788e4105f8622b68.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [57]:
# Creating data collator

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Evaluating model

In [58]:
# Import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [59]:
# Define an evaluation function to pass into trainer later
def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=1)

  return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

### Applying untrained model to text

In [60]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
  
  # tokenize text
  inputs = tokenizer.encode(text, return_tensors="pt")
  
  # compute logits
  logits = model(inputs).logits
  
  # convert logits to label
  predictions = torch.argmax(logits)

  print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - Negative
Not a fan, don't recommed. - Negative
Better than the first one. - Negative
This is not worth watching even once. - Negative
This one is a pass. - Negative


## Model Training

### Configuring LORA

In [61]:
peft_config = LoraConfig(
  task_type="SEQ_CLS", # Sequence classification
  r=4, # Intrictic rank of trainable weiht matrix
  lora_alpha=32, # This is like a learning rate
  lora_dropout=0.01, # Probability of dropout (zero-ing random weights)
  target_modules = ['q_lin'] # Which layers do we apply LORA to
)

peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, base_model_name_or_path=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules=['q_lin'], lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True)

In [62]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1221124 || all params: 67584004 || trainable%: 1.8068239934408148


###  Configuring Trainer

In [63]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 1

In [64]:
# define training arguments
training_args = TrainingArguments(
  output_dir= model_checkpoint + "-lora-text-classification",
  learning_rate=lr,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  num_train_epochs=num_epochs,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
)

# creater trainer object
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_dataset["train"],
  eval_dataset=tokenized_dataset['test'],
  tokenizer=tokenizer,
  data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
  compute_metrics=compute_metrics,
)

### Training model

In [65]:
# train model
trainer.train()

  0%|          | 1/250 [01:01<4:17:12, 61.98s/it]

KeyboardInterrupt: 